#CLD Score-based Generative modeling with the architecture given on the [paper github](https://github.com/nv-tlabs/CLD-SGM)

In [ ]:
! git clone https://github.com/nv-tlabs/CLD-SGM.git

In [ ]:
! pip install configargparse
! pip install torchdiffeq
! pip install ninja

In [ ]:
import torch
device = 'cuda'
print(torch.cuda.is_available())

True


In [ ]:
%load_ext autoreload
%autoreload 2

import configargparse
import json
import sde_lib
import sampling
import util.utils as utils
import util.checkpoint as checkpoint
from util import datasets
from models import ncsnpp
import models.utils as mutils
from models.ema import ExponentialMovingAverage
from torchvision.utils import make_grid
import numpy as np
import matplotlib.pyplot as plt
import likelihood
import gc
%matplotlib inline

In [ ]:
p = configargparse.ArgParser()
p.add('-cc', is_config_file=True, default='default_cifar10.txt')
p.add('-sc', is_config_file=True, default='specific_cifar10.txt')

p.add('--root', default='.')
p.add('--workdir', default='work_dir')
p.add('--eval_folder', default=None)
p.add('--mode', choices=['train', 'eval', 'continue'], default='eval')
p.add('--cont_nbr', type=int, default=None)
p.add('--checkpoint', default=None)

p.add('--n_gpus_per_node', type=int, default=1)
p.add('--n_nodes', type=int, default=1)
p.add('--node_rank', type=int, default=0)
p.add('--master_address', default='127.0.0.1')
p.add('--master_port', type=int, default=6020)
p.add('--distributed', action='store_false')

p.add('--overwrite', action='store_true')

p.add('--seed', type=int, default=0)

# Data
p.add('--dataset')
p.add('--is_image', action='store_true')
p.add('--image_size', type=int)
p.add('--center_image', action='store_true')
p.add('--image_channels', type=int)
p.add('--data_dim', type=int)  # Dimension of non-image data
p.add('--data_location', default=None)

# SDE
p.add('--sde')
p.add('--beta_type')
# Linear beta params
p.add('--beta0', type=float)
p.add('--beta1', type=float)
# ULD params
p.add('--m_inv', type=float)
p.add('--gamma', type=float)
p.add('--numerical_eps', type=float)

# Optimization
p.add('--optimizer')
p.add('--learning_rate', type=float)
p.add('--weight_decay', type=float)
p.add('--grad_clip', type=float)

# Objective
p.add('--cld_objective', choices=['dsm', 'hsm'], default='hsm')
p.add('--loss_eps', type=float)
p.add('--weighting', choices=['likelihood', 'reweightedv1', 'reweightedv2'])

# Model
p.add('--name')
p.add('--ema_rate', type=float)
p.add('--normalization')
p.add('--nonlinearity')
p.add('--n_channels', type=int)
p.add('--ch_mult')
p.add('--n_resblocks', type=int)
p.add('--attn_resolutions')
p.add('--resamp_with_conv', action='store_true')
p.add('--use_fir', action='store_true')
p.add('--fir_kernel')
p.add('--skip_rescale', action='store_true')
p.add('--resblock_type')
p.add('--progressive')
p.add('--progressive_input')
p.add('--progressive_combine')
p.add('--attention_type')
p.add('--init_scale', type=float)
p.add('--fourier_scale', type=int)
p.add('--conv_size', type=int)
p.add('--dropout', type=float)
p.add('--mixed_score', action='store_true')
p.add('--embedding_type', choices=['fourier', 'positional'])

# Training
p.add('--training_batch_size', type=int)
p.add('--testing_batch_size', type=int)
p.add('--sampling_batch_size', type=int)
p.add('--n_train_iters', type=int)
p.add('--n_warmup_iters', type=int)
p.add('--snapshot_freq', type=int)
p.add('--log_freq', type=int)
p.add('--eval_freq', type=int)
p.add('--likelihood_freq', type=int)
p.add('--fid_freq', type=int)
p.add('--eval_threshold', type=int, default=1)
p.add('--likelihood_threshold', type=int, default=1)
p.add('--snapshot_threshold', type=int, default=1)
p.add('--fid_threshold', type=int, default=1)
p.add('--fid_samples_training', type=int)
p.add('--n_eval_batches', type=int)
p.add('--n_likelihood_batches', type=int)
p.add('--autocast_train', action='store_true')
p.add('--save_freq', type=int, default=None)
p.add('--save_threshold', type=int, default=1)

# Sampling
p.add('--sampling_method', choices=['ode', 'em', 'sscs'], default='ode')
p.add('--sampling_solver', default='scipy_solver')
p.add('--sampling_solver_options', type=json.loads, default={'solver': 'RK45'})
p.add('--sampling_rtol', type=float, default=1e-5)
p.add('--sampling_atol', type=float, default=1e-5)
p.add('--sscs_num_stab', type=float, default=0.)
p.add('--denoising', action='store_true')
p.add('--n_discrete_steps', type=int)
p.add('--striding', choices=['linear', 'quadratic', 'logarithmic'], default='linear')
p.add('--sampling_eps', type=float)

# Likelihood
p.add('--likelihood_solver', default='scipy_solver')
p.add('--likelihood_solver_options', type=json.loads, default={'solver': 'RK45'})
p.add('--likelihood_rtol', type=float, default=1e-5)
p.add('--likelihood_atol', type=float, default=1e-5)
p.add('--likelihood_eps', type=float, default=1e-5)
p.add('--likelihood_hutchinson_type', choices=['gaussian', 'rademacher'], default='rademacher')

# Evaluation
p.add('--ckpt_file')
p.add('--eval_sample', action='store_true')
p.add('--autocast_eval', action='store_true')
p.add('--eval_loss', action='store_true')
p.add('--eval_fid', action='store_true')
p.add('--eval_likelihood', action='store_true')
p.add('--eval_fid_samples', type=int, default=50000)
p.add('--eval_jacobian_norm', action='store_true')
p.add('--eval_iw_likelihood', action='store_true')
p.add('--eval_density', action='store_true')
p.add('--eval_density_npts', type=int, default=101)
p.add('--eval_sample_hist', action='store_true')
p.add('--eval_hist_samples', type=int, default=100000)
p.add('--eval_loss_variance', action='store_true')
p.add('--eval_loss_variance_images', type=int, default=1)
p.add('--eval_sample_samples', type=int, default=1)

batch_size = 16
config = p.parse_args(args=['--distributed',
                            '--training_batch_size', str(batch_size),
                            '--testing_batch_size', str(batch_size),
                            '--sampling_batch_size', str(batch_size)])

config.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
inverse_scaler = utils.get_data_inverse_scaler(config)

def plot_samples(x):
    nrow = int(np.sqrt(x.shape[0]))
    image_grid = make_grid(inverse_scaler(x).clamp(0., 1.), nrow)
    plt.axis('off')
    plt.imshow(image_grid.permute(1, 2, 0).cpu())

In [ ]:
beta_fn = utils.build_beta_fn(config)
beta_int_fn = utils.build_beta_fn(config)
sde = sde_lib.CLD(config, beta_fn, beta_int_fn)

In [ ]:
import losses
loss_CLD = losses.get_loss_fn(sde,True,config)

In [ ]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.optim import Adam

# score_model = torch.nn.DataParallel(ncsnpp.NCSNpp(config))
# score_model = score_model.to(device)

score_model = mutils.create_model(config).to(config.device)
score_model = torch.nn.DataParallel(score_model)
optim_params = score_model.parameters()
optimizer = utils.get_optimizer(config, optim_params)

n_epochs = 10
## size of a mini-batch
batch_size =  32
## learning rate
lr=2e-4

dataset = MNIST('.', train=True, transform=transforms.ToTensor(), download=True)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

optimizer = Adam(score_model.parameters(), lr=lr)

In [ ]:
#Training
import tqdm
tqdm_epoch = tqdm.notebook.trange(n_epochs)
for epoch in tqdm_epoch:
  total_loss = []
  avg_loss = 0.
  num_items = 0
  for x, y in data_loader:
    x = x.to(device)    
    loss = loss_CLD(score_model, x)
    loss = torch.mean(loss)
    total_loss.append(loss.detach().cpu().numpy())
    optimizer.zero_grad()
    loss.backward()    
    optimizer.step()
    avg_loss += loss.item() * x.shape[0]
    num_items += x.shape[0]
  plt.plot(total_loss)
  plt.show()
  # Print the averaged training loss so far.
  tqdm_epoch.set_description('Average Loss: {:5f}'.format(avg_loss / num_items))
  # Update the checkpoint after each epoch of training.
  torch.save(score_model.state_dict(), 'ckpt.pth')